<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/GatheringBaseMetrics_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gathering Metrics

# Setup env


In [0]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [0]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 34
-rw------- 1 root root 21727 Jun  4 19:53 data_prep.py
drwx------ 2 root root  4096 May 10 13:31 __pycache__
-rw------- 1 root root  8416 Jun  2 21:24 visualizations.py


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [0]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [0]:
# install required libs
!pip install mido

     |████████████████████████████████| 61kB 4.2MB/s 


In [0]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep as dp

In [0]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [12]:
# testing auto reload of modules 
dp.test_function_call('bling')

Test function called worked! when: 2020-06-11 14:05:13.399089,  param:bling


## Pandas display options

In [0]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Load some data

In [13]:

file_1 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'
file_7 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer7/eval_session/1_funk-groove1_138_beat_4-4.mid'

file_df, f, mtt = dp.load_file(file_7)  # load data

FILE name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer7/eval_session/1_funk-groove1_138_beat_4-4.mid
    loaded file: <midi file '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer7/eval_session/1_funk-groove1_138_beat_4-4.mid' type 0, 1 tracks, 521 messages>
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    track count: 1, tracks: [<midi track 'Midi Drums' 521 messages>]
    MIDI file type: 0
    > processing track: <midi track 'Midi Drums' 521 messages>
    note_on span - first tick: 0 , last tick: 30493 
    good instruments: 4, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 49.0: 'Crash Cymbal 1 (49)', 51.0: 'Ride Cymbal 1 (51)'}
    bar info - bars in file: 16, bar quantize level: 16.0
    tick info - ticks per time sig beat: 480.0, ticks per quantize bin: 120


In [14]:
file_df.head()
file_df.info()

track_msg_num  total_ticks  note  velocity  \
bar_number bar_beat_number note                                              
1          1               36             0:6            0    36        47   
                           51             0:7            0    51       113   
           3               51            0:10          241    51        55   
                           36            0:11          245    36        43   
           5               38            0:14          505    38       108   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          1               36              0           0           1   
                           51              0           0           1   
           3               51              1         240           1   
                           36              5         240           1   
           5               38             25         480           1   

                                 bar_beat_number  
bar_number bar_beat_number note                   
1          1               36                  1  
                           51                  1  
           3               51                  3  
                           36                  3  
           5               38                  5

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 257 entries, (1, 1, 36) to (16, 15, 51)
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   track_msg_num    257 non-null    string  
 1   total_ticks      257 non-null    int64   
 2   note             257 non-null    int64   
 3   velocity         257 non-null    int64   
 4   beat_offset      257 non-null    int64   
 5   beat_center      257 non-null    category
 6   bar_number       257 non-null    int64   
 7   bar_beat_number  257 non-null    int64   
dtypes: category(1), int64(6), string(1)
memory usage: 16.5 KB


In [0]:
tmp_df = file_df.copy()


# GroupBy tests

Very good info on using DataFrame groupby operations...
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
* https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
* https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
* https://chrisalbon.com/python/data_wrangling/pandas_apply_operations_to_groups/
* https://stackoverflow.com/a/51231000
* http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/



In [0]:
# calculate number of hits per metric position 
op = tmp_df.groupby(level=1)['note'].count()  

In [23]:
print(str(op))

bar_beat_number
1     28
2      4
3     32
4      4
5     32
7     16
8     16
9     16
10    12
11    32
12     8
13    25
15    24
16     8
Name: note, dtype: int64


We have duplicate columns & index names, need to drop some stuff to explore more.

In [0]:
tmp2 = tmp_df.drop(columns=['bar_beat_number', 'note'])

In [28]:
tmp2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 257 entries, (1, 1, 36) to (16, 15, 51)
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   track_msg_num  257 non-null    string  
 1   total_ticks    257 non-null    int64   
 2   velocity       257 non-null    int64   
 3   beat_offset    257 non-null    int64   
 4   beat_center    257 non-null    category
 5   bar_number     257 non-null    int64   
dtypes: category(1), int64(4), string(1)
memory usage: 12.5 KB


In [0]:
# calculate number of hits per instrument, per metric position 
op2 = tmp2.groupby(by=['bar_beat_number', 'note'])['velocity'].count()

In [62]:
op2

bar_beat_number  note
1                36      12
                 51      16
2                38       4
3                36      16
                 51      16
4                36       4
5                38      16
                 51      16
7                51      16
8                38      16
9                51      16
10               38      12
11               36      16
                 49       4
                 51      12
12               36       8
13               38       8
                 51      17
15               38       8
                 51      16
16               38       8
Name: velocity, dtype: int64

In [0]:
op3 = pd.DataFrame(op2)

In [0]:
op3.columns = ['hit_count']

# Frequency table

Handy to know..
* https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [69]:
op3

hit_count
bar_beat_number note           
1               36           12
                51           16
2               38            4
3               36           16
                51           16
4               36            4
5               38           16
                51           16
7               51           16
8               38           16
9               51           16
10              38           12
11              36           16
                49            4
                51           12
12              36            8
13              38            8
                51           17
15              38            8
                51           16
16              38            8

In [0]:
# flip things around, fill with '0' instead of NaN, then convert to int
op4 = op3.unstack(0).fillna(0).astype('int32')

In [76]:
op4

hit_count                                                
bar_beat_number        1  2   3  4   5   7   8   9   10  11 12  13  15 16
note                                                                     
36                     12  0  16  4   0   0   0   0   0  16  8   0   0  0
38                      0  4   0  0  16   0  16   0  12   0  0   8   8  8
49                      0  0   0  0   0   0   0   0   0   4  0   0   0  0
51                     16  0  16  0  16  16   0  16   0  12  0  17  16  0

# Detect multiple hits in quantize bucket

This detects multiple hits of the same instrument, within our quantize window - this is a problem. Need to decide what to do here, maybe ignore all these hits by pre-filtering?

In [0]:
m_hits = file_df.copy()

In [0]:
m_hits.drop(columns=['note'], inplace=True)

In [99]:
m_hits.groupby(['beat_center','note']).count().head(20)

track_msg_num  total_ticks  velocity  beat_offset  \
beat_center note                                                      
0           36              1.0          1.0       1.0          1.0   
            38              NaN          NaN       NaN          NaN   
            49              NaN          NaN       NaN          NaN   
            51              1.0          1.0       1.0          1.0   
120         36              NaN          NaN       NaN          NaN   
            38              NaN          NaN       NaN          NaN   
            49              NaN          NaN       NaN          NaN   
            51              NaN          NaN       NaN          NaN   
240         36              1.0          1.0       1.0          1.0   
            38              NaN          NaN       NaN          NaN   
            49              NaN          NaN       NaN          NaN   
            51              1.0          1.0       1.0          1.0   
360         36              NaN          NaN       NaN          NaN   
            38              NaN          NaN       NaN          NaN   
            49              NaN          NaN       NaN          NaN   
            51              NaN          NaN       NaN          NaN   
480         36              NaN          NaN       NaN          NaN   
            38              1.0          1.0       1.0          1.0   
            49              NaN          NaN       NaN          NaN   
            51              1.0          1.0       1.0          1.0   

                  bar_number  bar_beat_number  
beat_center note                               
0           36           1.0              1.0  
            38           NaN              NaN  
            49           NaN              NaN  
            51           1.0              1.0  
120         36           NaN              NaN  
            38           NaN              NaN  
            49           NaN              NaN  
            51           NaN              NaN  
240         36           1.0              1.0  
            38           NaN              NaN  
            49           NaN              NaN  
            51           1.0              1.0  
360         36           NaN              NaN  
            38           NaN              NaN  
            49           NaN              NaN  
            51           NaN              NaN  
480         36           NaN              NaN  
            38           1.0              1.0  
            49           NaN              NaN  
            51           1.0              1.0

Handy filter functions..
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#filtration

In [102]:
m_hits.groupby(['beat_center','note']).filter(lambda a_group: len(a_group) > 1)

track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
10         13              51           0:323        18761        55   
                           51           0:324        18764       127   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
10         13              51             41       18720          10   
                           51             44       18720          10   

                                 bar_beat_number  
bar_number bar_beat_number note                   
10         13              51                 13  
                           51                 13